In [9]:
%pip install numpy
%pip install tensorflow
%pip install opencv-python
%pip install sklearn-python
%pip install matplotlib
%pip install keras
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: unknown command "insteall" - maybe you meant "install"



In [6]:
import numpy as np
import os
import tensorflow as tf
import cv2
from keras.metrics import OneHotMeanIoU
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tqdm import tqdm 
from skimage.io import imread, imshow
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from glob import glob
from multi_unet import Multiple_UNet
from metrics import dice_coef, dice_loss

ModuleNotFoundError: No module named 'numpy'

In [ ]:
color_mapping_text = {
  (128, 64, 128): "road",
  (0, 0, 142): "car",
  (0, 0, 70): "truck",
  (0, 60, 100): "bus",
  (220, 20, 60): "person",
  (70, 70, 70): "building",
  (250, 170, 30): "traffic light",
  (153, 153, 153): "pole",
  (102, 102, 156): "wall",
  (190, 153, 153): "fence",
  (220, 220, 0): "signboard",
  (107, 142, 35): "vegetation",
  (244, 35, 232): "sidewalk",
  (152, 251, 152): "terrain",
  (70, 130, 180): "sky",
  (0, 0, 0): "unnamed"
}

color_mapping = {
  (128, 64, 128): 0,
  (0, 0, 142): 1,
  (0, 0, 70): 2,
  (0, 60, 100): 3,
  (220, 20, 60): 4,
  (70, 70, 70): 5,
  (250, 170, 30): 6,
  (153, 153, 153): 7,
  (102, 102, 156): 8,
  (190, 153, 153): 9,
  (220, 220, 0): 10,
  (107, 142, 35): 11,
  (244, 35, 232): 12,
  (152, 251, 152): 13,
  (70, 130, 180): 14,
  (0, 0, 0): 15
}

inverse_mapping = {value: key for key, value in color_mapping.items()}
inverse_mapping.keys()


dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
def rgb_to_one_hot(rgb_image, color_mapping):
    """
    Convert RGB image to one-hot encoded image based on color mapping.
    
    Args:
        rgb_image (numpy.ndarray): Input RGB image array.
        color_mapping (dict): Dictionary mapping RGB tuples to class indices.
        
    Returns:
        numpy.ndarray: One-hot encoded image array.
    """
    one_hot_image = np.zeros((rgb_image.shape[0], rgb_image.shape[1], len(color_mapping)), dtype=np.uint8)
    
    for rgb_tuple, class_index in color_mapping.items():
        mask = np.all(rgb_image == np.array(rgb_tuple), axis=-1)
        one_hot_image[..., class_index] = mask
    
    return one_hot_image

In [ ]:


def one_hot_to_rgb(one_hot_image, inverse_mapping, default_value=(0, 0, 0)):
  """
  Decode one-hot encoded image to RGB image using inverse color mapping,
  processing each 16-element vector individually.

  Args:
    one_hot_image (numpy.ndarray): Input one-hot encoded image array.
    inverse_mapping (dict): Dictionary mapping class indices to RGB tuples.
    default_value (tuple, optional): Default RGB value to assign for missing classes. Defaults to (0, 0, 0) - black.

  Returns:
    numpy.ndarray: Decoded RGB image array.
  """

  if len(one_hot_image.shape) != 3 or one_hot_image.shape[-1] != len(inverse_mapping):
    raise ValueError("One-hot image dimensions or channel count mismatch")

  rgb_image = np.empty((one_hot_image.shape[0], one_hot_image.shape[1], 3), dtype=np.uint8)

  for row in range(one_hot_image.shape[0]):
    for col in range(one_hot_image.shape[1]):
      one_hot_vector = one_hot_image[row, col, :]

      class_index = np.argmax(one_hot_vector)
      if class_index in inverse_mapping:
          rgb_value = inverse_mapping[class_index]
      else:
          rgb_value = default_value

      rgb_image[row, col, :] = rgb_value

  return rgb_image


In [ ]:
IMAGE_DIR = 'dataset/images'
MASK_DIR = 'dataset/segments'

H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data():

  images = sorted(glob(os.path.join(IMAGE_DIR, "*.jpg")))
  masks = sorted(glob(os.path.join(MASK_DIR, "*.png")))
  
  return images, masks


def read_image(path):
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x = x.astype(np.float16)
  return x


def read_mask(path):
  x = cv2.imread(path, cv2.IMREAD_COLOR)
  x = cv2.resize(x, (W, H))
  x = x / 255.0
  x = x.astype(np.float16)
  return x


def display_image_mask(image_path, mask_path):
  image = read_image(image_path)
  mask = read_mask(mask_path)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
  ax1.imshow(image)
  ax1.set_title('Image')
  ax1.axis('off')
  ax2.imshow(mask, cmap='gray')
  ax2.set_title('Mask')
  ax2.axis('off')
  plt.tight_layout()
  plt.show()


In [ ]:
images, masks = load_data()

X = []
Y_pre = []

for image_path, mask_path in zip(images, masks):
    image = read_image(image_path)
    mask = read_mask(mask_path)
    X.append(image)
    Y_pre.append(mask)

X = np.array(X)
Y_pre = np.array(Y_pre)

In [ ]:
Y = []
for mask in Y_pre:
    element = rgb_to_one_hot(mask, color_mapping)
    Y.append(element)

Y = np.array(Y)

In [ ]:
print(X.shape, Y.shape)

(7470, 256, 256, 3) (7470, 256, 256, 16)


In [ ]:
batch_size = 8
num_epochs = 10
validation_split = 0.1
np.random.seed = 32
lr = 1e-5
model_path = os.path.join("files", "multiple.keras")
csv_path = os.path.join("files", "log.csv")

In [ ]:
model = Multiple_UNet()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 256,  │          0 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │      2,320 │ dropout[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128, 128,  │          0 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │      9,248 │ dropout_1[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64, 64,    │          0 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     36,928 │ dropout_2[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32, 32,    │          0 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 32, 32,    │    147,584 │ dropout_3[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 16, 16,    │          0 │ conv2d_7[0][0]  

 Total params: 1,941,360 (7.41 MB)

 Trainable params: 1,941,360 (7.41 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr), loss=[dice_loss], metrics=[OneHotMeanIoU(num_classes=16)])

checkpointer = tf.keras.callbacks.ModelCheckpoint('models/checkpoint.keras', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.CSVLogger(csv_path)
]

In [ ]:
history = model.fit(X, Y, validation_split=validation_split , epochs=num_epochs, callbacks= callbacks, batch_size= batch_size)

Epoch 1/10
 79/841 ━━━━━━━━━━━━━━━━━━━━ 7:43 609ms/step - loss: 0.9833 - one_hot_mean_io_u: 2.2488e-06

In [ ]:
model.save('models/multiple.keras')

In [ ]:
import random

idx = random.randint(0, len(X))

test_image = X[idx]

print(test_image.shape)

ground_truth = Y_pre[idx]

print(ground_truth.shape)

In [ ]:
test_image = test_image.reshape((H, W, 3))
test_preds = model.predict(test_image)
test_preds.shape

In [ ]:
test_preds_img = one_hot_to_rgb(test_preds, inverse_mapping)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

test_image = test_image.reshape((H, W, 3))
test_preds_img = test_preds_img.reshape((H, W, 3))
ground_truth = ground_truth.reshape((H, W, 3))


axs[0].imshow(test_image)
axs[0].set_title('Test Image')
axs[0].axis('off')

axs[1].imshow(test_preds_img, cmap='gray')
axs[1].set_title('Predicted Mask')
axs[1].axis('off')

axs[2].imshow(ground_truth)  # Random ground truth mask
axs[2].set_title('Ground Truth Mask')
axs[2].axis('off')

plt.tight_layout()
plt.show()